In [1]:
import mne
import pandas as pd
import os

import matplotlib.pyplot as plt


import numpy as np
import platform

import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

from base.files_in_out import GetFiles
from stats.stats_helper import filter_list, tTest_ana, filter_list_equal
import stats.stats_helper as s_hp


In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [122]:
g_num=None
eeg_format='-ave.fif'
eeg_exp='tsk'
datafolder='ana/MNE/evo_list'


#files = getListOfFiles(datafolder,g_num)

files = GetFiles(datafolder,g_num=g_num,eeg_format=eeg_format)

evo_files=files.fflist

# Correlation RT and ERPs


## Load df with RT

In [96]:
# Load RT df
path='ana/behavioral/df_o_cleantrials.feather'
RT_df=pd.read_feather(path)

In [97]:
# fitler only normal correct trials and end of the diastole
RT_df=RT_df[RT_df['difficulty']== 'normal']

RT_df=RT_df[RT_df['sys_mask']== 1]
RT_df=RT_df[RT_df['accuracy']== 'correct']

## Define parameters for ERPs


In [98]:
picks_P1=['D32','A11','A12','A13','A14','A24','A25','A26','A27','B8','B9','B10']


picks_P2=['A13','A14','A15','A23','A24','A25','A26','A27','A28']

picks_VAN=['D32','A9','A10','A11','A12','A16','A15','A14','A13',
           'A22','A23','A24','A25','A29','A28','A27','A26','B6','B7','B8','B9','B10']

picks_P3a=['C11','C12','C13','C14','C15','C18','C19','C20','C21','C22','C25','C26','C27','C28','C32','D4']

picks_P3b=['A2','A3','A4','A19','A20','A21','A22','A32','A31','A30','A29','A5','A18',
          'A17','A16','A8','A9','A6','A7','D29','D30','D15','D16','D17','D28','D27']

picks_dict=dict(p1=picks_P1,p2=picks_P2,VAN=picks_VAN,p3a=picks_P3a,p3b=picks_P3b)
time_int_dict= dict(p1=[0.09,0.12],p2=[0.14,0.16],VAN=[0.24,0.32 ],p3a=[0.32,0.36],p3b=[0.4,0.5])

## Load ERP for condition

In [136]:
phy_cond='sys'
awa_cond='aware'
rsp_cond='o'
if rsp_cond=='o':
    rsp_cond_lab='tsk_o_'
elif rsp_cond=='n':
    rsp_cond_lab=''
cond=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{awa_cond}_{phy_cond}_list'))


ana/MNE/evo_list/maskON/mistake/nc/xns/norm/maskON_mistake_nc_xns_norm_unaware_sys_list-ave.fif
Reading ana/MNE/evo_list/maskON/correct/cfa/vep/norm/tsk_o_maskON_correct_cfa_vep_norm_aware_sys_list-ave.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.42 × vep/normal/correct/aware/sys/exh + 0.58 × vep/normal/correct/aware/sys/inh
 G_n= g01)
        0 CTF compensation matrices available
        nave = 95 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.40 × vep/normal/correct/aware/sys/exh + 0.60 × vep/normal/correct/aware/sys/inh
 G_n= g03)
        0 CTF compensation matrices available
        nave = 121 - aspect type = 100
Projections

In [137]:
# Main loop to extract average potential around defined compoenents
dict_df={}
#loop around dict with picked electrodes
for k,item in picks_dict.items():
    #predefine list for g numbers and amplitude data
    list_g_num=[]
    list_avg_data=[]

    for erp_g in cond:
        #store info on time interval
        time_int=time_int_dict[k]
        #extract both time interval and electrodes
        erp_data=erp_g.copy().crop(time_int[0],time_int[1]).pick_channels(item).data
        #compute average from filtered ERP data
        avg_data=np.average(erp_data)
        #append data to list
        list_avg_data.append(avg_data)
        # append g_num to list
        g_num=erp_g.comment[-3:]
        list_g_num.append(g_num)
    #assign complete list to dict with componenet as key
    dict_df[k]=list_avg_data
# assign g_num list to dict
dict_df['g_num']=list_g_num
dict_df

{'p1': [-9.628389042726226e-07,
  7.361367679245042e-06,
  -3.2051142547907652e-06,
  -9.523147973574716e-07,
  5.396972721764352e-06,
  3.408530608708218e-07,
  3.386605109135922e-06,
  4.384336648399767e-06,
  4.879754864283035e-06,
  7.23424185990864e-06,
  4.828490605535552e-06,
  4.841203350565067e-06,
  5.361794525737407e-07,
  3.7430010076409693e-06,
  1.926477953721699e-06,
  3.0463326649062857e-06,
  2.0467772694816916e-06,
  4.36361194812286e-06,
  -1.0322786216931945e-06,
  9.27247215490415e-06,
  1.3109710359843824e-06,
  5.9535982704464574e-06,
  8.034454244954793e-06,
  7.946773796045173e-06,
  1.5506097980323965e-06,
  3.772162752001828e-06,
  4.159766836698543e-06,
  5.191268169508274e-06,
  1.313906085758592e-05,
  4.395978449437358e-06],
 'p2': [2.9318927122151054e-06,
  7.826538148105169e-06,
  -2.0819276205271727e-07,
  2.5891667529317936e-06,
  -8.108459484093247e-09,
  4.180624545016245e-06,
  8.534281564438802e-06,
  3.4192119623863044e-06,
  7.880736823743198e-0

In [138]:
#create pd dataframe from dict
pd_df=pd.DataFrame.from_dict(dict_df)

In [139]:
pd_df

,p1,p2,VAN,p3a,p3b,g_num
0,-9.628389e-07,2.931893e-06,-9.667400e-07,-8.049808e-07,1.587457e-06,g01
1,7.361368e-06,7.826538e-06,-9.759335e-07,3.413963e-07,-2.606751e-08,g03
2,-3.205114e-06,-2.081928e-07,1.000709e-06,-4.082294e-06,5.252905e-06,g08
3,-9.523148e-07,2.589167e-06,1.295697e-06,3.659024e-08,4.853533e-07,g10
4,5.396973e-06,-8.108459e-09,-1.311562e-06,1.171954e-06,2.955179e-06,g11
5,3.408531e-07,4.180625e-06,-5.617171e-07,-6.773648e-07,1.915894e-06,g12
6,3.386605e-06,8.534282e-06,1.219063e-08,-1.811056e-06,2.281106e-06,g15
7,4.384337e-06,3.419212e-06,-1.604334e-06,-2.580755e-06,2.002415e-06,g16
8,4.879755e-06,7.880737e-06,-8.509932e-06,5.923362e-06,7.082539e-07,g17
9,7.234242e-06,-2.286545e-06,-7.095457e-06,1.322903e-06,4.781290e-07,g19


## Filter RT df and create df with averaged RT for each subject

In [140]:
# this code filters df based on the preselected condition (used also to load erps to avoid issues)
if awa_cond !='':
    RT_df_cond=RT_df[RT_df['awareness']==awa_cond]
    print(f'includes only {awa_cond}')
else:
    print(f'includes all awareness')

if phy_cond!='':
    if phy_cond =='sys' or phy_cond =='dia':
        RT_df_cond=RT_df[RT_df['cardiac_phase']==phy_cond]
        print(f'includes only {phy_cond}')
    else:
        RT_df_cond=RT_df[RT_df['rsp_phase']==phy_cond]
        print(f'includes only {phy_cond}')
else:
    print(f'includes all phy cond')


pivot_RT_df=pd.pivot_table(RT_df_cond,index=['g_num'],values='RT')
pivot_RT_df

includes only aware
includes only sys


,RT
g_num,
g01,762.496588
g03,853.517749
g08,849.946961
g10,872.992785
g11,590.471765
g12,719.516085
g15,792.158826
g16,638.724513
g17,837.130160


## Combine ERP and RT df to correlate

In [141]:

final_df=pivot_RT_df.merge(pd_df,on='g_num')
final_df

,g_num,RT,p1,p2,VAN,p3a,p3b
0,g01,762.496588,-9.628389e-07,2.931893e-06,-9.667400e-07,-8.049808e-07,1.587457e-06
1,g03,853.517749,7.361368e-06,7.826538e-06,-9.759335e-07,3.413963e-07,-2.606751e-08
2,g08,849.946961,-3.205114e-06,-2.081928e-07,1.000709e-06,-4.082294e-06,5.252905e-06
3,g10,872.992785,-9.523148e-07,2.589167e-06,1.295697e-06,3.659024e-08,4.853533e-07
4,g11,590.471765,5.396973e-06,-8.108459e-09,-1.311562e-06,1.171954e-06,2.955179e-06
5,g12,719.516085,3.408531e-07,4.180625e-06,-5.617171e-07,-6.773648e-07,1.915894e-06
6,g15,792.158826,3.386605e-06,8.534282e-06,1.219063e-08,-1.811056e-06,2.281106e-06
7,g16,638.724513,4.384337e-06,3.419212e-06,-1.604334e-06,-2.580755e-06,2.002415e-06
8,g17,837.130160,4.879755e-06,7.880737e-06,-8.509932e-06,5.923362e-06,7.082539e-07
9,g19,657.792286,7.234242e-06,-2.286545e-06,-7.095457e-06,1.322903e-06,4.781290e-07


In [142]:
# compute corr using pearson
final_df.corr()

,RT,p1,p2,VAN,p3a,p3b
RT,1.000000,0.108562,0.369507,-0.079320,-0.029969,-0.145837
p1,0.108562,1.000000,0.150934,-0.299799,-0.056224,-0.153188
p2,0.369507,0.150934,1.000000,-0.008740,0.119952,-0.222390
VAN,-0.079320,-0.299799,-0.008740,1.000000,-0.523074,0.229226
p3a,-0.029969,-0.056224,0.119952,-0.523074,1.000000,-0.493689
p3b,-0.145837,-0.153188,-0.222390,0.229226,-0.493689,1.000000


In [143]:
# compute corr using spearman
final_df.corr(method='spearman')

,RT,p1,p2,VAN,p3a,p3b
RT,1.000000,-0.059844,0.378865,0.081201,0.080311,-0.277864
p1,-0.059844,1.000000,0.206674,-0.356174,-0.036263,-0.043826
p2,0.378865,0.206674,1.000000,0.010456,0.035818,-0.283204
VAN,0.081201,-0.356174,0.010456,1.000000,-0.448721,0.149277
p3a,0.080311,-0.036263,0.035818,-0.448721,1.000000,-0.481646
p3b,-0.277864,-0.043826,-0.283204,0.149277,-0.481646,1.000000


In [144]:
# code to obtain p values
from scipy.stats import pearsonr,spearmanr
import pandas as pd

def calculate_pvalues_pearson(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(pearsonr(tmp[r], tmp[c])[1], 4)
    return pvalues

def calculate_pvalues_spearman(df):
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            tmp = df[df[r].notnull() & df[c].notnull()]
            pvalues[r][c] = round(spearmanr(tmp[r], tmp[c])[1], 4)
    return pvalues

In [145]:

final_df_nog=final_df.drop('g_num',1)
calculate_pvalues_pearson(final_df_nog)

C:\Users\Vivi\AppData\Local\Temp\ipykernel_19188\2212112361.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  final_df_nog=final_df.drop('g_num',1)


,RT,p1,p2,VAN,p3a,p3b
RT,0.0,0.568,0.0445,0.6769,0.8751,0.4419
p1,0.568,0.0,0.4259,0.1075,0.7679,0.419
p2,0.0445,0.4259,0.0,0.9634,0.5278,0.2375
VAN,0.6769,0.1075,0.9634,0.0,0.003,0.223
p3a,0.8751,0.7679,0.5278,0.003,0.0,0.0056
p3b,0.4419,0.419,0.2375,0.223,0.0056,0.0


In [146]:
calculate_pvalues_spearman(final_df_nog)

,RT,p1,p2,VAN,p3a,p3b
RT,0.0,0.7534,0.039,0.6697,0.6731,0.1371
p1,0.7534,0.0,0.2732,0.0534,0.8491,0.8181
p2,0.039,0.2732,0.0,0.9563,0.851,0.1294
VAN,0.6697,0.0534,0.9563,0.0,0.0129,0.4311
p3a,0.6731,0.8491,0.851,0.0129,0.0,0.007
p3b,0.1371,0.8181,0.1294,0.4311,0.007,0.0


# Compute nasal vs oral on P1 amplitude
  - load oral and nasal for awa and unaware ERPs
  - extract P1 electrodes and time window (90-120 ms)
  - compute Anovas